<a href="https://colab.research.google.com/github/ashiqurrahmankhan21st/BreastCancer/blob/main/Breast_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
#import keras_metrics
from keras.utils import to_categorical
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import RocCurveDisplay
from sklearn.inspection import PartialDependenceDisplay
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import time

In [179]:
df = pd.read_csv("https://raw.githubusercontent.com/ashiqurrahmankhan21st/BreastCancer/main/data.csv")
del df['id']
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [55]:
df['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [58]:
encoder = LabelEncoder()
df['diagnosis'] = encoder.fit_transform(df['diagnosis'])
df['diagnosis'].value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

In [66]:
X = df.drop(columns=['diagnosis']).copy()
y = df.diagnosis.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=123)
print("Original data : ",df.shape)
print("tarin         : ",X_train.shape)
print("test          : ",X_test.shape[0])
print("validation    : ",X_val.shape[0])

Original data :  (569, 31)
tarin         :  (455, 30)
test          :  57
validation    :  57


In [22]:
# SVM

In [60]:
svm = make_pipeline(StandardScaler(), SVC(gamma='auto', kernel = 'rbf'))
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
print("train accuracy",svm.score(X_train, y_train))
print("test accuracy",svm.score(X_test, y_test))

train accuracy 0.9868131868131869
test accuracy 0.9824561403508771


In [61]:
y_pred_svm

array([0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1])

In [ ]:
#SVM
svm = make_pipeline(StandardScaler(), SVC(gamma='auto', kernel = 'rbf'))
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
print("train accuracy",svm.score(X_train, y_train))
print("test accuracy",svm.score(X_test, y_test))

In [144]:
#ANN
model = Sequential()
model.add(Dense(30, activation='selu', input_shape=(X_train.shape[1],)))
model.add(Dense(10, activation='selu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='BinaryCrossentropy', optimizer='rmsprop', metrics=['accuracy']) #tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
start_time = time.time()
model.fit(X_train,y_train, batch_size = 128, epochs = 50, validation_data = (X_val,y_val))
end_time = time.time()
y_pred_ANN = (model.predict(X_test) > 0.5).astype("int32")
ATr = model.evaluate(X_train,y_train,verbose=0)#[1]
ATe = model.evaluate(X_test,y_test,verbose=0)#[1]

Epoch 1/50
4/4 [==============================] - 1s 78ms/step - loss: 70.0751 - accuracy: 0.3758 - val_loss: 58.0119 - val_accuracy: 0.3860
Epoch 2/50
4/4 [==============================] - 0s 14ms/step - loss: 55.5642 - accuracy: 0.3758 - val_loss: 47.2455 - val_accuracy: 0.3860
Epoch 3/50
4/4 [==============================] - 0s 14ms/step - loss: 45.3071 - accuracy: 0.3758 - val_loss: 38.4470 - val_accuracy: 0.3860
Epoch 4/50
4/4 [==============================] - 0s 15ms/step - loss: 36.5602 - accuracy: 0.3758 - val_loss: 30.3256 - val_accuracy: 0.3860
Epoch 5/50
4/4 [==============================] - 0s 14ms/step - loss: 28.4029 - accuracy: 0.3758 - val_loss: 22.2183 - val_accuracy: 0.3860
Epoch 6/50
4/4 [==============================] - 0s 15ms/step - loss: 19.8025 - accuracy: 0.3758 - val_loss: 13.5784 - val_accuracy: 0.3860
Epoch 7/50
4/4 [==============================] - 0s 13ms/step - loss: 11.1677 - accuracy: 0.3758 - val_loss: 5.5392 - val_accuracy: 0.4211
Epoch 8/50
4/4

In [145]:
print(ATr[1])
print(ATe[1])

0.9274725317955017
0.9473684430122375


In [152]:
#XGBoost
params = {
            'objective':'binary:logistic',
            'max_depth': 4,
            'alpha': 10,
            'learning_rate': 1.0,
            'n_estimators':100
        }
xgb = XGBClassifier(**params)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
print('XGBoost model train accuracy score: {0:0.4f}'. format(accuracy_score(y_train, xgb.predict(X_train))))
print('XGBoost model test accuracy score: {0:0.4f}'. format(accuracy_score(y_test, xgb.predict(X_test))))

XGBoost model train accuracy score: 0.9890
XGBoost model test accuracy score: 0.9825


In [164]:
newdata = pd.DataFrame({
    "SVM": y_pred_svm
})
newdata["ANN"] = y_pred_ANN
newdata["XGB"] = y_pred_xgb
newdata['y_test'] = list(y_test)
newdata

,SVM,ANN,XGB,y_test
0,0,0,0,0
1,0,0,0,0
2,1,1,1,1
3,1,1,1,1
4,1,1,1,1
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0
